Partical Component Analysis
========

In [144]:
import pandas as pd
import numpy as np

In [145]:
D=pd.read_excel("http://data.hossam.kr/E04/boston.xlsx")

In [146]:
D.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CRIM       506 non-null    float64
 1   ZN         506 non-null    float64
 2   INDUS      506 non-null    float64
 3   CHAS       506 non-null    int64  
 4   NOX        506 non-null    float64
 5   RM         506 non-null    float64
 6   AGE        506 non-null    float64
 7   DIS        506 non-null    float64
 8   RAD        506 non-null    int64  
 9   TAX        506 non-null    int64  
 10  PTRATIO    506 non-null    float64
 11  B          506 non-null    float64
 12  LSTAT      506 non-null    float64
 13  MEDV       506 non-null    float64
 14  CAT. MEDV  506 non-null    int64  
dtypes: float64(11), int64(4)
memory usage: 59.4 KB


## PCA 에 대한 과정에 대해 알아보자

In [147]:
data=pd.DataFrame(D)
y=data["MEDV"]
x=data[data.columns.difference(["MEDV"])]
x=x.drop(columns=['CAT. MEDV',"CHAS"],axis=1)

### step1. 공분산 행렬부터 구하기

In [148]:
import numpy as np
import scipy as sp
import sklearn.preprocessing as skpre

#### 공분산 행렬을 구하기 이전에 전부 min_max scale과 z-score scale을 진행하겠음

In [149]:
# 여기서 주의할 점은 PCA 검사는 이상치에 영향을 많이 받을 수 밖에 없다
pre_x=x.apply(skpre.minmax_scale) #min_max
pre_x=x.apply(skpre.scale) #z-score

#### 공분산 행렬 구하기

In [150]:
# 직접계산
corr_matrix=pre_x.T@pre_x/len(pre_x)

In [151]:
pre_x.corr(method='pearson')

,AGE,B,CRIM,DIS,INDUS,LSTAT,NOX,PTRATIO,RAD,RM,TAX,ZN
AGE,1.000000,-0.273534,0.352734,-0.747881,0.644779,0.602339,0.731470,0.261515,0.456022,-0.240265,0.506456,-0.569537
B,-0.273534,1.000000,-0.385064,0.291512,-0.356977,-0.366087,-0.380051,-0.177383,-0.444413,0.128069,-0.441808,0.175520
CRIM,0.352734,-0.385064,1.000000,-0.379670,0.406583,0.455621,0.420972,0.289946,0.625505,-0.219247,0.582764,-0.200469
DIS,-0.747881,0.291512,-0.379670,1.000000,-0.708027,-0.496996,-0.769230,-0.232471,-0.494588,0.205246,-0.534432,0.664408
INDUS,0.644779,-0.356977,0.406583,-0.708027,1.000000,0.603800,0.763651,0.383248,0.595129,-0.391676,0.720760,-0.533828
LSTAT,0.602339,-0.366087,0.455621,-0.496996,0.603800,1.000000,0.590879,0.374044,0.488676,-0.613808,0.543993,-0.412995
NOX,0.731470,-0.380051,0.420972,-0.769230,0.763651,0.590879,1.000000,0.188933,0.611441,-0.302188,0.668023,-0.516604
PTRATIO,0.261515,-0.177383,0.289946,-0.232471,0.383248,0.374044,0.188933,1.000000,0.464741,-0.355501,0.460853,-0.391679
RAD,0.456022,-0.444413,0.625505,-0.494588,0.595129,0.488676,0.611441,0.464741,1.000000,-0.209847,0.910228,-0.311948
RM,-0.240265,0.128069,-0.219247,0.205246,-0.391676,-0.613808,-0.302188,-0.355501,-0.209847,1.000000,-0.292048,0.311991


#### 고유값 분해
numpy 를 이용하면 쉽게 할 수 있음

In [152]:
eigen_value,eigen_vector=np.linalg.eig(corr_matrix)

#### eigen_value와 eigen_vactor 정렬

In [153]:
eigen_value

array([6.1267188 , 1.34247929, 1.17978483, 0.83506595, 0.66467141,
       0.53740698, 0.39641745, 0.06464564, 0.27706753, 0.22026544,
       0.1693152 , 0.18616148])

In [154]:
#정렬을 해줬다는걸 알수있지?
sort_ls=[(x,y) for x,y  in enumerate(eigen_value)]
sort_ls.sort(key=lambda x:x[1],reverse=True)
sort_ls

[(0, 6.126718796151475),
 (1, 1.342479287575696),
 (2, 1.179784833120062),
 (3, 0.8350659474013684),
 (4, 0.6646714055381014),
 (5, 0.5374069846919036),
 (6, 0.39641744958183045),
 (8, 0.2770675290521341),
 (9, 0.2202654428851525),
 (11, 0.18616147945240596),
 (10, 0.16931519958396934),
 (7, 0.06464564496590483)]

#### 위의 결과를 통한 주성분 결정

#### 제 1 주성분

In [170]:
pc1=pre_x@eigen_vector[0]*eigen_value[0]
pc1

0     -2.973397
1     -7.630666
2     -9.317063
3     -8.004482
4     -8.830923
         ...   
501   -3.845866
502   -3.054554
503   -7.137883
504   -6.250566
505   -3.159797
Length: 506, dtype: float64

In [168]:
pc2=pre_x@eigen_vector[1]*eigen_value[1]
pc2

0      1.211675
1      0.457248
2      0.358049
3      0.668794
4      0.806971
         ...   
501    0.243842
502    0.240031
503    0.505200
504    0.433823
505    0.191780
Length: 506, dtype: float64

### 모듈을 이용해 구해보자

In [157]:
from sklearn.decomposition import PCA

In [158]:
len(pre_x.columns)

12

In [162]:
model=PCA(n_components=2)

In [163]:
fit=model.fit_transform(pre_x)

In [164]:
fit

array([[-2.09723388, -0.72017904],
       [-1.456003  , -0.94769427],
       [-2.07345404, -0.62533575],
       ...,
       [-0.31128314, -1.39524641],
       [-0.26939083, -1.35059004],
       [-0.12452556, -1.34293829]])